# VDOT basics

VDOT is a system devised by Jack Daniels and Jimmy Gilbert
that aims to capture runners' physiological capabilities and
provide them with suitable workout intensity paces.
It was first introduced in their book _Oxygen Power (1979)_,
and is used proeminently in Daniels' more recent _Daniels' Running Formula_.

This notebook is a quick exploration of the concepts surrounding VDOT.

**The notebook uses math based on the formulas and methodology presented in _Oxygen Power_.**
The results will not match up perfectly with Daniels' most recent tables or his
[online calculator](https://vdoto2.com/) as the "formula" has changed without
its details being made public.

## Introduction

The basis of VDOT calculation revolves around the relationship of two curves:
a _running economy_ curve that relates oxygen consumption to running velocity,
and a curve relating a race's duration with the typical fraction of
maximal oxygen consumption a runner will perform at for that duration.
With these curves a _performance indicator_ value can be derived
from a runner's race performance.

The relationship between oxygen consumption $\mathrm{\dot{V}O_2}$
($\mathrm{ml \cdot kg^{-1} \cdot min^{-1}}$)
and running velocity $\mathrm{v}$
($\mathrm{m \cdot min^{-1}}$) is given by:

$$
\mathrm{\dot{V}O_2} = -4.60 + 0.182258 \mathrm{v} + 0.000104 \mathrm{v^2}
$$

<img src="images/equation-1.png" width=500>

The relationship between the fraction of $\mathrm{\dot{V}O_2max}$
and the duration of a distance race $\mathrm{t}$ ($\mathrm{min}$) is given by:

$$
\mathrm{F\dot{V}O_2max} = 0.8 + 0.1894393 e^{-0.012778 \mathrm{t}} + 0.2989558 e^{-0.1932605 \mathrm{t}}
$$

<img src="images/equation-2.png" width=650>

VDOT (pseudo $\mathrm{\dot{V}O_2max}$) can then be easily calculated as:

$$
\mathrm{VDOT} = \frac{\mathrm{\dot{V}O_2}}{\mathrm{F\dot{V}O_2max}}
$$

---

Below is the $\mathrm{VDOT}$ calculation of a runner that ran a **5k race in 24 minutes and 34 seconds**.

In [1]:
import math
from datetime import timedelta

d = 5000
t = timedelta(minutes=24, seconds=34).total_seconds() / 60
v = d / t

vo2 = -4.6 + 0.182258 * v + 0.000104 * v**2
pct = 0.8 + 0.1894393 * math.exp(-0.012778 * t) + 0.2989558 * math.exp(-0.1932605 * t)

vdot = vo2 / pct
vdot

39.11039839652726

Looks about right from inspecting Daniels' $\mathrm{VDOT}$ tables:

| VDOT | 1,500 | Mile | 3,000 | 2 mile | 5,000 | 10K   | 15K   | Half marathon | Marathon |
| ---- | ----- | ---- | ----- | ------ | ----- | ----- | ----- | ------------- | -------- |
| ...  |
| 38   | 6:54  | 7:27 | 14:41 | 15:49  | **25:12** | 52:17 | 80:33 | 1:55:55       | 3:59:35  |
| 39   | 6:44  | 7:17 | 14:21 | 15:29  | **24:39** | 51:09 | 78:47 | 1:53:24       | 3:54:34  |
| 40   | 6:35  | 7:07 | 14:03 | 15:08  | **24:08** | 50:03 | 77:06 | 1:50:59       | 3:49:45  |
| ...  |

## Estimating other race performances

The formulas presented above can be used to estimate various race times
once a VDOT value has been calculated.

This can be done by finding the root of $f(d, t)$ (while setting $d$ to the desired race distance):

$$
f(d, t) = \frac{-4.6 + 0.182258 d t^{-1} + 0.000104 d^2 t^{-2}}{0.8 + 0.1894393 e^{-0.012778 \mathrm{t}} + 0.2989558 e^{-0.1932605 \mathrm{t}}} - \mathrm{VDOT}
$$

This is a root finding problem of a non-linear equation that can be solved with iterative methods.
For simplicy, let's use the bisection method:

In [2]:
from scipy import optimize


def format_duration(td):
    s = int(td.total_seconds())
    hh, mm, ss = s // 3600, s // 60 % 60, s % 60

    if hh > 0:
        return f"{hh}:{mm:02}:{ss:02}"

    return f"{mm}:{ss:02}"


def f(x, vdot, d):
    return (-4.6 + 0.182258 * d * x ** (-1) + 0.000104 * d**2 * x ** (-2)) / (
        0.8 + 0.1894393 * math.exp(-0.012778 * x) + 0.2989558 * math.exp(-0.1932605 * x)
    ) - vdot


distances = {
    "5K": 5000,
    "10K": 10000,
    "Half marathon": 21097.5,
    "Marathon": 42195,
}

for name, d in distances.items():
    root = optimize.bisect(f, 1, 600, args=(vdot, d))
    td = timedelta(minutes=root)

    td_s = format_duration(td)
    print(f"{name:14}: {td_s}")

5K            : 24:34
10K           : 50:58
Half marathon : 1:53:02
Marathon      : 3:53:54


These results deviate a couple of seconds from the official online calculator,
but then again, so do Daniels' tables.

## Defining training paces

The calculation of training paces is not explained or talked about in _Oxygen Power_.
However, in _Chapter 4: Training Runs and Itensities_ of _Daniels' Running Formula_,
some intensity ranges (based on percentages of a runner's $\mathrm{\dot{V}O_2max}$)
are set for different types of running workouts:

- Easy: 59-74%
- Marathon: 75-84%
- Threshold: 83-88%
- Interval: 95-100%
- Repetition: 105-120%

Using the above defined relationship between velocity and oxygen consumption,
the pace ranges can be calculated:

$$
\mathrm{v} = \frac{-0.182258 + \sqrt{0.033218 - 0.000416 (-4.6 -\mathrm{\dot{V}O_2})}}{0.000208}
$$

In [3]:
def g(vdot, pct):
    return (
        -0.182258 + math.sqrt(0.033218 - 0.000416 * (-4.6 - (vdot * pct)))
    ) / 0.000208


paces = {
    "Easy": (0.59, 0.74),
    "Marathon": (0.75, 0.84),
    "Threshold": (0.83, 0.88),
    "Interval": (0.95, 1),
    "Repetition": (1.05, 1.2),
}

for name, pcts in paces.items():
    velocities = [g(vdot, pct) for pct in pcts]  # meters/min
    paces = [1000 / v for v in velocities]  # min/km
    paces_s = [format_duration(timedelta(minutes=p)) for p in paces]  # "m:ss" format

    slower, faster = paces_s
    print(f"{name:11}: {faster} ~ {slower}")

Easy       : 5:57 ~ 7:06
Marathon   : 5:22 ~ 5:53
Threshold  : 5:11 ~ 5:26
Interval   : 4:40 ~ 4:52
Repetition : 4:02 ~ 4:29


The "official" values (and ranges) for this VDOT are generally close
to the above intervals; I guess that's something.

Seriously though, it seems this approach doesn't wield as results
as good as the race estimations.
A closer look at the tables and the pace progression is needed to improve here.